In [2]:
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np

def plot_spectrogram(sample_rate, audio_data, filename):
    frequencies, times, Sxx = signal.spectrogram(audio_data, sample_rate, window='hann')
    plt.figure()
    plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx + 1e-10)) 
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.yscale('log')
    plt.colorbar(label='Intensity [dB]')
    plt.savefig('spectrogram_' + filename + '.png')
    plt.clf()
    return Sxx, frequencies, times

def apply_savitzky_golay(audio_data, window_length, polyorder):
    return signal.savgol_filter(audio_data, window_length, polyorder)

def apply_butterworth(audio_data, cutoff_freq, sample_rate, order=5):
    nyquist = 0.5 * sample_rate
    normal_cutoff = cutoff_freq / nyquist
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    return signal.filtfilt(b, a, audio_data)

def apply_wiener(audio_data, mysize=29, noise=None):
    return signal.wiener(audio_data, mysize=mysize, noise=noise)

def detect_peaks(Sxx, frequencies, times, time_delta=0.1, freq_delta=50):
    peaks = []
    for i in range(len(frequencies)):
        for j in range(len(times)):
            if Sxx[i, j] == Sxx[i, max(0, j - 1): min(len(times), j + 2)].max():
                if Sxx[i, j] == Sxx[max(0, i - 1): min(len(frequencies), i + 2), j].max():
                    peaks.append((times[j], frequencies[i]))
    return peaks

def main():
    sample_rate, audio_data = wav.read('music.wav')
    audio_data = audio_data.flatten()  # Ensure it is a 1D array
    plot_spectrogram(sample_rate, audio_data, 'original')

    audio_savgol = apply_savitzky_golay(audio_data, window_length=101, polyorder=3)
    audio_butter = apply_butterworth(audio_savgol, cutoff_freq=1000, sample_rate=sample_rate, order=3)
    audio_wiener = apply_wiener(audio_butter)

    plot_spectrogram(sample_rate, audio_savgol, 'filtered_savgol')
    plot_spectrogram(sample_rate, audio_butter, 'filtered_butter')
    plot_spectrogram(sample_rate, audio_wiener, 'filtered_wiener')

    wav.write('filtered_savgol.wav', sample_rate, audio_savgol.astype(np.int16))
    wav.write('filtered_butter.wav', sample_rate, audio_butter.astype(np.int16))
    wav.write('filtered_wiener.wav', sample_rate, audio_wiener.astype(np.int16))

    Sxx, frequencies, times = plot_spectrogram(sample_rate, audio_wiener, 'denoised')
    peaks = detect_peaks(Sxx, frequencies, times)

    with open('peaks.txt', 'w') as f:
        for peak in peaks:
            f.write(f"Time: {peak[0]}, Frequency: {peak[1]}\n")

if __name__ == "__main__":
    main()


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>